1. 문서 내용을 읽음
2. 문서를 쪼갬
    * 토큰수 초과로 답변을 생성하지 못할 수 있음
    * 문서가 길면(input) 답변 생성이 오래걸림
3. embedding -> vector database 로 저장
4. 질문이 있을 때, 백터 데이터베이스 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [9]:
%pip install --upgrade --quiet docx2txt langchain-community langchain_text_splitters langchain_chroma load_dotenv langchain_openai



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter( 
    chunk_size=1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader('./study-sources/tax_docs/tax_with_table.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [10]:
from dotenv import load_dotenv 
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embedding = OpenAIEmbeddings(model='text-embedding-3-large')

In [11]:
from langchain_chroma import Chroma

#database = Chroma.from_documents(documents=document_list, embedding=embeddings)
database = Chroma.from_documents(collection_name='chroma-tax', persist_directory="./chroma", embedding=embedding, documents=document_list)

In [12]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
retrieved_docs = database.similarity_search(query, k=3)

retrieved_docs

[Document(metadata={'source': './study-sources/tax_docs/tax_with_table.docx'}, page_content='나. 그 밖의 배당소득에 대해서는 100분의 14\n\n3. 원천징수대상 사업소득에 대해서는 100분의 3. 다만, 외국인 직업운동가가 한국표준산업분류에 따른 스포츠 클럽 운영업 중 프로스포츠구단과의 계약(계약기간이 3년 이하인 경우로 한정한다)에 따라 용역을 제공하고 받는 소득에 대해서는 100분의 20으로 한다.\n\n4. 근로소득에 대해서는 기본세율. 다만, 일용근로자의 근로소득에 대해서는 100분의 6으로 한다.\n\n5. 공적연금소득에 대해서는 기본세율\n\n5의2.제20조의3제1항제2호나목 및 다목에 따른 연금계좌 납입액이나 운용실적에 따라 증가된 금액을 연금수령한 연금소득에 대해서는 다음 각 목의 구분에 따른 세율. 이 경우 각 목의 요건을 동시에 충족하는 때에는 낮은 세율을 적용한다.\n\n가. 연금소득자의 나이에 따른 다음의 세율\n\n\n\n나. 삭제<2014. 12. 23.>\n\n다. 사망할 때까지 연금수령하는 대통령령으로 정하는 종신계약에 따라 받는 연금소득에 대해서는 100분의 4\n\n5의3. 제20조의3제1항제2호가목에 따라 퇴직소득을 연금수령하는 연금소득에 대해서는 다음 각 목의 구분에 따른 세율. 이 경우 연금 실제 수령연차 및 연금외수령 원천징수세율의 구체적인 내용은 대통령령으로 정한다.\n\n가. 연금 실제 수령연차가 10년 이하인 경우: 연금외수령 원천징수세율의 100분의 70\n\n나. 연금 실제 수령연차가 10년을 초과하는 경우: 연금외수령 원천징수세율의 100분의 60\n\n6. 기타소득에 대해서는 다음에 규정하는 세율. 다만, 제8호를 적용받는 경우는 제외한다.\n\n가. 제14조제3항제8호라목 및 마목에 해당하는 소득금액이 3억원을 초과하는 경우 그 초과하는 분에 대해서는 100분의 30\n\n나. 제21조제1항제18호 및 제21호에 따른 기타소득에 

In [13]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4o')

In [14]:
%pip install -U langchain langchainhub --quiet


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [15]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

/Users/heyjohnis/.pyenv/versions/3.11.11/lib/python3.11/site-packages/langsmith/client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [16]:
from langchain.chains import RetrievalQA

database = Chroma(collection_name='chroma-tax', persist_directory="./chroma", embedding_function=embedding)

qa_chain = RetrievalQA.from_chain_type(
    llm, 
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

In [17]:
ai_message = qa_chain({"query": query})

/var/folders/3y/c7gj2cyj10q50qg744vxt7t40000gp/T/ipykernel_58509/3455095564.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  ai_message = qa_chain({"query": query})


In [18]:
ai_message

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': '소득세를 계산하기 위해서는 근로소득에 대한 기본세율을 적용해야 합니다. 그러나 제공된 정보에는 구체적인 기본세율이 명시되어 있지 않으므로 정확한 소득세 금액을 계산할 수 없습니다. 정확한 소득세를 알기 위해서는 세율표와 공제 항목을 참조해야 합니다.'}